In [2]:
# load packages
import pandas as pd
import numpy as np
from fbprophet import Prophet
import os

import warnings
from stldecompose import decompose, forecast
from stldecompose.forecast_funcs import (naive, drift, mean, seasonal_naive)

In [3]:
def run_prophet(series, timeframe):
    """
    Runs the Prophet 
    
    Key arguments:
    --------------
    series -- (DataFrame) time series data
    timeframe -- (DataFrame) a DataFrame with one column 
                 consisting of predicted dates

    Returns: 
    --------------
    Returns the forecast of the predictions 

    """
    model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False, 
                    # changepoint_prior_scale=0.001,
                    # mcmc_samples=300,
                    interval_width=0.95)
    model.fit(series)
    forecast = model.predict(timeframe)
    return forecast, model

In [4]:
# read data
data = pd.read_csv("../data/exception_hours.csv", parse_dates=["SHIFT_DATE"])

In [5]:
# clean data
data_clean = data[(data["SITE"]=="St Paul's Hospital") |
                    (data["SITE"]=="Mt St Joseph") |
                    (data["SITE"]=="Holy Family") |
                    (data["SITE"]=="SVH Langara") |
                    (data["SITE"]=="Brock Fahrni") |
                    (data["SITE"]=="Youville Residence")]
data_clean = data_clean[(data_clean["JOB_FAMILY"]=="DC1000") |
                    (data_clean["JOB_FAMILY"]=="DC2A00") |
                    (data_clean["JOB_FAMILY"]=="DC2B00") ]

In [6]:
# create training dataframes
data_group = data_clean.groupby(["JOB_FAMILY", "SITE", "SUB_PROGRAM", "SHIFT_DATE"]).size().reset_index()
data_group = data_group.rename({"SHIFT_DATE":"ds", 0:"y"}, axis=1)

In [3]:
# create timeframe data for prediction
timeframe_past = pd.DataFrame(pd.date_range(start='2013-01-01', end='2018-12-31', freq="D")).rename({0:"ds"}, axis=1)
timeframe_future = pd.DataFrame(pd.date_range(start='2019-01-01', end='2019-12-31', freq="D")).rename({0:"ds"}, axis=1)

In [15]:
import datetime
start = pd.to_datetime("2019-12-31")-datetime.timedelta(days=4*365)
end = pd.to_datetime("2019-12-31")

In [8]:
sites = data_clean["SITE"].unique()
job_families = data_clean["JOB_FAMILY"].unique()
sub_programs = data_clean["SUB_PROGRAM"].unique()

In [71]:
warnings.simplefilter('ignore')
# create and store predictions and true results
models = {}
data_individual = {}
pred_results = {}
for i in sites:
    for j in job_families:
        for k in sub_programs:
            temp_data = data_group[(data_group["SITE"]==i) & (data_group["JOB_FAMILY"]==j) & (data_group["SUB_PROGRAM"]==k)].reset_index()
            temp_data = pd.merge(timeframe_past, temp_data, on="ds", how="outer")
            temp_data["y"] = temp_data["y"].fillna(0)

            if temp_data["y"].sum() >= 300.0:
                data_individual[(i, j, k)] = temp_data
                pred_results[(i, j, k)], models[(i, j, k)] = run_prophet(temp_data, timeframe_future)
                print("Fitting -", i, j, k, ": Done")

Fitting - St Paul's Hospital DC2B00 RENAL 6AB : Done
Fitting - St Paul's Hospital DC2B00 OR PAR SPH : Done
Fitting - St Paul's Hospital DC2B00 HEMODIALYSIS 6CD IC : Done
Fitting - St Paul's Hospital DC2B00 PASU 9A 2N 8C : Done
Fitting - St Paul's Hospital DC2B00 EMERG SPH : Done
Fitting - St Paul's Hospital DC2B00 ICU : Done
Fitting - St Paul's Hospital DC2B00 NICU MATERNITY : Done
Fitting - St Paul's Hospital DC2B00 MELANIE MULDER : Done
Fitting - St Paul's Hospital DC2B00 MED NURSE RELIEF : Done
Fitting - St Paul's Hospital DC2B00 IMAGING : Done
Fitting - St Paul's Hospital DC2B00 RENAL CDU IAMHD : Done
Fitting - St Paul's Hospital DC2B00 10C, CLINICS : Done
Fitting - St Paul's Hospital DC2B00 AMBULATORY : Done
Fitting - St Paul's Hospital DC2B00 BARBARA HALL : Done
Fitting - St Paul's Hospital DC2B00 NURSING UNIT SPH : Done
Fitting - St Paul's Hospital DC2B00 PALLIATIVE SRVCS : Done
Fitting - St Paul's Hospital DC2B00 DIONNE NORDBY : Done
Fitting - St Paul's Hospital DC2B00 CLINICS 

In [75]:
weekly = {}
for i in data_individual.keys():
    # create week column
    combined= pred_results[i][["ds", "yhat", "yhat_lower", "yhat_upper"]]
    combined["week"] = combined["ds"].dt.week
    combined["ds"] = combined["ds"]-pd.DateOffset(weekday=0, weeks=1)

    # store y, yhat, yhat_lower, yhat_upper
    weekly_yhat = combined.groupby("ds").yhat.sum().round(0).astype(int).reset_index()
    weekly_yhat_lower = combined.groupby("ds").yhat_lower.sum().round(0).astype(int).reset_index()
    weekly_yhat_upper = combined.groupby("ds").yhat_upper.sum().round(0).astype(int).reset_index()

    # merge weekly results
    weekly[i] = pd.concat([weekly_yhat["yhat"],
                           weekly_yhat_lower["yhat_lower"],
                           weekly_yhat_upper["yhat_upper"]], axis=1)

    # create columns "year", "site", "JOB_FAMILY"
    length = weekly[i].shape[0]
    weekly[i]["ds"] = combined["ds"]
    weekly[i]["week"] = combined["ds"].dt.weekofyear
    weekly[i]["site"] = np.repeat(i[0], length)
    weekly[i]["job_family"] = np.repeat(i[1], length)
    weekly[i]["sub_program"] = np.repeat(i[2], length)

    # code for minimizing errors (model residuals)
    forecasted = models[i].predict(timeframe_future)
    actual = data_individual[i]

    # get residuals
    error = actual["y"] - forecasted["yhat"]
    obs = timeframe_past.copy()
    obs["error"] = error
    obs = obs.set_index("ds")

    # model residuals
    period = int((np.max(timeframe_future) - np.min(timeframe_future)).dt.days)+1
    decomp = decompose(obs, period=period)
    weekly_fcast = forecast(decomp, steps=period, fc_func=drift, seasonal=True)
    weekly_fcast["week"] = weekly_fcast.index-pd.DateOffset(weekday=0, weeks=1)
    weekly_fcast = weekly_fcast.groupby("week").sum()

    # replace weekly data
    resid_fcast = weekly_fcast.reset_index()["drift+seasonal"]
    weekly_yhat = (weekly[i]["yhat"] + resid_fcast).round(0)
    weekly_yhat_lower = (weekly[i]["yhat_lower"] + resid_fcast).round(0)
    weekly_yhat_upper = (weekly[i]["yhat_upper"] + resid_fcast).round(0)

    # replace negatives with 0s
    weekly[i]["yhat"] = weekly_yhat.where(weekly_yhat >= 0, 0)
    weekly[i]["yhat_lower"] = weekly_yhat_lower.where(weekly_yhat_lower >= 0, 0)
    weekly[i]["yhat_upper"] = weekly_yhat_upper.where(weekly_yhat_upper >= 0, 0)

In [76]:
# create data/predictions folder if it doesn't exist
predictions_path = "../data/predictions/"
if not os.path.exists(predictions_path):
    os.mkdir(predictions_path)


# export to "data/predictions/" directory
total_data = pd.DataFrame()
for i in weekly:
    total_data = pd.concat([total_data, weekly[i]], axis=0)
total_data.to_csv(predictions_path + "predictions.csv")